 Imports

In [1]:
import torch
from transformers import LEDTokenizer, LEDForConditionalGeneration, AutoTokenizer
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import datetime
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
from tqdm.auto import tqdm
%load_ext autoreload

2023-11-15 12:45:11.767726: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-15 12:45:11.960993: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-15 12:45:13.549457: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-11-15 12:45:13.549563: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
import utils.pdf_reader as pdfr

In [3]:
from transformers import (
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
)

In [4]:
from datasets import load_dataset, load_metric

In [5]:
!nvidia-smi

Wed Nov 15 12:45:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.85.02    Driver Version: 510.85.02    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A40          Off  | 00000000:21:00.0 Off |                    0 |
|  0%   63C    P0   204W / 300W |  39313MiB / 46068MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A40          Off  | 00000000:81:00.0 Off |                    0 |
|  0%   

 Load Model and Data

In [6]:
tokenizer = AutoTokenizer.from_pretrained("allenai/led-base-16384")

In [7]:
train_dataset = load_dataset("narrativeqa", split="train[:1000]")

Found cached dataset narrativeqa (/home/vijays2/.cache/huggingface/datasets/narrativeqa/default/0.0.0/daef7ccc51ec258bef464658d11751bb20f033da9b4c219fd84563b3a4af0422)


In [8]:
val_dataset = load_dataset("narrativeqa", split="validation[:500]") # run some tests on a smaller sample size for now

Found cached dataset narrativeqa (/home/vijays2/.cache/huggingface/datasets/narrativeqa/default/0.0.0/daef7ccc51ec258bef464658d11751bb20f033da9b4c219fd84563b3a4af0422)


In [9]:
encoder_max_len = 16384
decoder_max_len = 512
batch_size = 2

In [10]:
data_dir = "./data"
log_dir = f"{data_dir}/experiments/LED/logs"
save_path = f"{data_dir}/experiments/LED/models/output.pt"
cache_path_train = f"{data_dir}/cache/LED.train"
cache_path_test = f"{data_dir}/cache/LED.test"

In [11]:
def encode(example,
           encoder_max_len=encoder_max_len, decoder_max_len=decoder_max_len):
  
    context = example['document']['text']
    question = example['question']['text']
    answer = example['answers'][0]['text']
  
    question_plus = f"answer_me: {str(question)}"
    question_plus += f" context: {str(context)} </s>"
    
    answer_plus = ', '.join([i for i in list(answer)])
    answer_plus = f"{answer_plus} </s>"
    
    encoder_inputs = tokenizer(question_plus, truncation=True, 
                               return_tensors='pt', max_length=encoder_max_len,
                              pad_to_max_length=True)
    
    decoder_inputs = tokenizer(answer_plus, truncation=True, 
                               return_tensors='pt', max_length=decoder_max_len,
                              pad_to_max_length=True)
    
    input_ids = encoder_inputs['input_ids'][0]
    input_attention = encoder_inputs['attention_mask'][0]
    target_ids = decoder_inputs['input_ids'][0]
    target_attention = decoder_inputs['attention_mask'][0]
    global_attention = torch.ones_like(input_attention)
    outputs = {'input_ids':input_ids, 'attention_mask': input_attention, 
               'labels':target_ids, 'decoder_attention_mask':target_attention, 'global_attention_mask':global_attention}
    return outputs



In [12]:
narrative_train = train_dataset.map(encode)

Loading cached processed dataset at /home/vijays2/.cache/huggingface/datasets/narrativeqa/default/0.0.0/daef7ccc51ec258bef464658d11751bb20f033da9b4c219fd84563b3a4af0422/cache-c13f570e2bc0a91e.arrow


In [13]:
narrative_val = val_dataset.map(encode)

Loading cached processed dataset at /home/vijays2/.cache/huggingface/datasets/narrativeqa/default/0.0.0/daef7ccc51ec258bef464658d11751bb20f033da9b4c219fd84563b3a4af0422/cache-73e902b5987dbd6a.arrow


In [14]:
def to_dataset(dataset):  
  columns = ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask']
  dataset.set_format(type='torch', columns=columns)
  return dataset
  


In [15]:
train_ds = to_dataset(narrative_train)
valid_ds = to_dataset(narrative_val)

In [16]:
import gc

gc.collect()
torch.cuda.empty_cache()

In [17]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
train_dataloader = DataLoader(
    train_ds, shuffle=True, batch_size=batch_size, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    valid_ds, batch_size=batch_size, collate_fn=data_collator
)

In [18]:
from accelerate import Accelerator
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

accelerator = Accelerator()
model = LEDForConditionalGeneration.from_pretrained("allenai/led-base-16384").half()
optimizer = AdamW(model.parameters(), lr=3e-5)
train_dl, eval_dl, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer
)

num_epochs = 5
num_training_steps = num_epochs * len(train_dl)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
iter = 0
for epoch in range(num_epochs):
    for batch in train_dl:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        if iter % 100 == 0: 
            torch.save(model.state_dict(), save_path)
            print("current loss: ",loss)
        iter += 1

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/home/vijays2/miniconda3/envs/tfv3/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/2500 [00:00<?, ?it/s]

You're using a LEDTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`

In [ ]:
model = LEDForConditionalGeneration.from_pretrained("allenai/led-base-16384").half().to("cuda:0")
model.load_state_dict(torch.load(save_path))

<All keys matched successfully>

In [ ]:
context = """We went on a trip to Europe. We had our breakfast at 7 am in the morning at \
the nearby coffee shop. Wore a dark blue over coat for our first visit to Louvre Museum \
to experience history and art."""

question = "Where did I go?"
print(context)
print(question)

We went on a trip to Europe. We had our breakfast at 7 am in the morning at the nearby coffee shop. Wore a dark blue over coat for our first visit to Louvre Museum to experience history and art.
Where did I go?


In [ ]:
input_text =  f"answer_me: {question} context: {context} </s>"
encoded_query = tokenizer(input_text, 
                         return_tensors='pt', pad_to_max_length=True, truncation=True, max_length=encoder_max_len).to("cuda:1")
input_ids = encoded_query["input_ids"].to("cuda:0")
attention_mask = encoded_query["attention_mask"].to("cuda:0")
global_attention_mask = torch.ones_like(attention_mask).to("cuda:0")
generated_answer = model.generate(input_ids, attention_mask=attention_mask, global_attention_mask=global_attention_mask,
                                 max_length=decoder_max_len, top_p=0.95, top_k=50, repetition_penalty=2.0)
decoded_answer = tokenizer.decode(generated_answer.cpu().numpy()[0])
print("Answer: ", decoded_answer)

/opt/conda/conda-bld/pytorch_1695392022560/work/aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [26,0,0], thread: [32,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1695392022560/work/aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [26,0,0], thread: [33,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1695392022560/work/aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [26,0,0], thread: [34,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1695392022560/work/aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [26,0,0], thread: [35,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1695392022560/work/aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [26,0,0], thread: [36,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/cond

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
